In [20]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [21]:
import pandas as pd

stores_df = pd.read_excel("https://data.gov.au/data/dataset/2fe5e2a9-8a3d-4dcf-baec-c5147d953150/resource/b59a15df-86ea-4c4c-95be-4dd9fc9f8ac4/download/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )
# stores_df = pd.read_excel("datasets/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )

stores_df = stores_df.iloc[: , :-1]
stores_df = stores_df.iloc[: , 1:]
stores_df = stores_df.melt(id_vars=["Year"], 
        var_name="country", 
        value_name="migration_no")
stores_df.rename(index=str, columns={"Year": "year"}, inplace=True)

stores_df.head()


,year,country,migration_no
0,Jan-Jun 1959,Australia,241.0
1,1959–60,Australia,674.0
2,1960–61,Australia,735.0
3,1961–62,Australia,745.0
4,1962–63,Australia,739.0


In [ ]:
stores_df.dropna(subset=['migration_no', 'country' ], inplace=True) 
stores_df['year'].replace(to_replace=".*([0-9]{4}).*", value=r"\1", regex=True, inplace=True)
stores_df = stores_df[stores_df.migration_no.astype('str').str.isnumeric()]

stores_df[['year','migration_no']] = stores_df[['year','migration_no']].astype(int)

stores_df = stores_df[~stores_df.country.str.contains('|'.join(['Australia', 'Other']))]

stores_df.dtypes